In [13]:
import nltk
import string

In [32]:
f = open('./text', 'r')
f2 = open('./result', 'r')
name = []

for line in f2:
    name.append(line.split('      ')[0])

keyword = ['brother']
person = ['my']
table = string.maketrans("","")       # remove the punctuations

for line in f:
    if person[0]+' '+keyword[0] in line:
        if line.split(person[0]+' '+keyword[0])[0].count('"')%2 == 1:
            if line.split(person[0]+' '+keyword[0])[1][0] == ' ':
                keyword_name = line.split(person[0]+' '+keyword[0])[1].split(' ')[1].translate(table, string.punctuation)
                if keyword_name in name:
                    print keyword_name
                    #part = line.split('"')
                    #for i in range(0,len(part),2):
                    #    print part[i]+'\n'
                    
f.close()
f2.close()

Edmure
Robb
Brandon
Rhaegar
Jaime
Jaime
Jaime
Rhaegar
Edmure
Jaime
Benjen
Jaime
Stannis
Brandon
